In [20]:
import pandas as pd
import ajuste_sen as aj
import numpy as np
from scipy.optimize import curve_fit
import altair as alt

In [21]:
def graph(data: pd.DataFrame) -> alt.Chart:
    return (
        alt.Chart(data)
        .mark_point(color="red")
        .encode(alt.X("freq", title="Frequência"), y="bode")
    )

#V_C/V_G
#V_R/V_G

In [22]:
frequencia_rc = [46.25,92.5,185,370,740,1480,2960,5920,11840]
frequencia_rl = [2375.375,4750.75,9501.5,19003,38006,76012,152024,304048,608096]
frequencia_rlc = [450,900,1800,3600,7200,14400,28800,57600]
amplitude_vg_rc = [2.5,2.5,2.5,2.49,2.48,2.46,2.46,2.45,2.46]
amplitude_vc_rc = [2.47,2.45,2.37,2.16,1.68,1.05,0.58,0.3,0.16]
amplitude_vg_rl = [2.27,2.27,2.28,2.29,2.3,2.36,2.44,2.47,2.46]
amplitdue_vl_rl = [0.12,0.17,0.28,0.53,0.98,1.62,2.16,2.38,2.43]
amplitude_vrc_rlc = [2.35,2.02,1.42,0.79,0.4,0.64,1.27,1.95,2.34]
amplitude_vg_rlc = [2.49,2.45,2.39,2.35,2.33,2.34,2.39,2.45,2.49]

In [23]:
'''def bode_freq(list_freq):
    data = pd.DataFrame()
    data["freq"] = list_freq
    lista_bode = []
    for freq in list_freq:
        lista_bode.append(20 * np.log10(2 * np.pi * freq))
    data["bode"] = lista_bode
    return data

graph(bode_freq(frequencia_rc)).display()
graph(bode_freq(frequencia_rl))'''

'def bode_freq(list_freq):\n    data = pd.DataFrame()\n    data["freq"] = list_freq\n    lista_bode = []\n    for freq in list_freq:\n        lista_bode.append(20 * np.log10(2 * np.pi * freq))\n    data["bode"] = lista_bode\n    return data\n\ngraph(bode_freq(frequencia_rc)).display()\ngraph(bode_freq(frequencia_rl))'

In [24]:
def bode_ganho(list_freq,A_vc,A_vg):
    data = pd.DataFrame()
    data["freq"] = list_freq
    lista_bode = []
    for i in range(len(list_freq)):
        lista_bode.append(20 * np.log10(A_vc[i]/A_vg[i]))
    data["bode"] = lista_bode
    return data

rc = graph(bode_ganho(frequencia_rc,amplitude_vc_rc,amplitude_vg_rc))
rl =graph(bode_ganho(frequencia_rl,amplitdue_vl_rl,amplitude_vg_rl))
rlc = graph(bode_ganho(frequencia_rlc,amplitude_vrc_rlc,amplitude_vg_rlc))

In [25]:
data = pd.read_csv("datas/bode rlc 20.csv")
data  = data.loc[data["Freq(Hz)"]<60000]

bode_real_rlc = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)", scale = alt.Scale(domain = [0,60000])),
    alt.Y("Gain(dB)")
)

data = pd.read_csv("datas/RL0.csv")
data = data.loc[data["Freq(Hz)"]<= 600000]

bode_real_rl = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)"),
    alt.Y("Gain(dB)",scale = alt.Scale(domain=[-25,0]))
)

data = pd.read_csv("datas/RC0.csv")
data = data.loc[data["Freq(Hz)"]<= 12000]


bode_real_rc = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)"),
    y = "Gain(dB)"
)

(bode_real_rlc + rlc).display()
(bode_real_rlc + rlc).save("ok.png")
(bode_real_rc + rc).display()
(bode_real_rl + rl).display()



alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)